In [1]:
from pyspark.sql import  SparkSession

In [2]:
spark = SparkSession.builder.appName('hacker').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/hack_data.csv', header = True, inferSchema = True)

In [4]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [5]:
data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

Out[ 9 ]: 
['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [9]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed']

In [10]:
assembler = VectorAssembler(inputCols= feat_cols, 
                            outputCol = 'features')

In [11]:
final_data = assembler.transform(data)

In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol = 'features',
                       outputCol = 'scaled')

In [14]:
final_data = scaler.fit(final_data).transform(final_data)

In [15]:
kmeans2 = KMeans(featuresCol= 'scaled', k=2)
kmeans3 = KMeans(featuresCol= 'scaled', k=3)

In [16]:
model_k2 = kmeans2.fit(final_data)
model_k3 = kmeans3.fit(final_data)

In [17]:
model_k3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 2| 79|
 0| 88|
+----------+-----+

In [18]:
model_k2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+